In [1]:
import time
import random
import os
import pandas as pd
import csv
import numpy as np
from itertools import chain
from pdp_lib import processing_google_map as proc
from pdp_lib import util
from GA_lib import GA_multi_depot as GA
from GA_lib import operation_multi_depot as operation
from GA_lib import evaluate_multi_depot as evaluate
from statistics import mode

In [12]:
def solve_and_print(filepath, rounds = 10, population_size = 100, generations=2000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    # DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    DEPOT_NUMBERS = proc.worse2worst_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    # DEPOT_NUMBERS = proc.KNN_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)

    print('----------------Instance name: ' + str(filepath) + '-----------------------------------')
    print(" processing time --- %s seconds ---" % (time.time() - process_time))

    total_results = []
    for round in range(rounds):
        results_all_depots = []
        distance_all_depots = []
        time_all_depots = []
        for id_depot in range(len(DEPOTS)):
            THIS_DEP_REQS = REQ_BY_DEPOTS[id_depot]
            NODES_OF_THIS_DEPOT = proc.locations_of_this_depot(id_depot, REQ_BY_DEPOTS, LOCATIONS)
            if(len(NODES_OF_THIS_DEPOT)==0):
                results_all_depots.append([])
                distance_all_depots.append(0)
                time_all_depots.append(0)
                continue
            # ############### SOLVING THE PROBLEMS !!!!!!!! ######################################
            pops_create_time = time.time()
            calculation_time = time.time()
            print('#################### Round :' +str(round+1)+' ############################')
            ## Initialize the populations
            populations = []
            for _ in range(population_size):
                chromosome = GA.initialize_Feasible_chromosome(DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS, LoadCapacities)
                populations.append(chromosome)
            # print("Populations creation time --- %s seconds ---" % (time.time() - pops_create_time))
            ## Crossovers and mutate ##
            GA_time = time.time()

            ## Evaluate fitnesses of all populations ##
            best_fitness_so_far = -99999999
            for gen in range(generations):
                fitness_table = []
                for chromosome in populations:
                    fitness_table.append(
                        evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS,
                                                   id_depot))
                populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
                populations.pop()
                populations.pop()
                elite1 = populations.pop(0)
                elite2 = populations.pop(0)


                ## Tournament Selection ##
                # id1, id2 = random.randrange(0, len(populations)), random.randrange(0, len(populations))
                id1,id2,id3,id4 = random.sample(range(0, len(populations)), 4)
                fitness1,fitness2,fitness3,fitness4 = fitness_table[id1],fitness_table[id2],fitness_table[id3],fitness_table[id4]
                candidate1,candidate2,candidate3,candidate4 = populations[id1], populations[id2],populations[id3],populations[id4]
                parent1 = operation.tournament_selection(candidate1,candidate2,fitness1,fitness2)
                parent2 = operation.tournament_selection(candidate3,candidate4,fitness3,fitness4)

                ## Crossing over !! ####
                child1, child2 = operation.crossover(DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, parent1, parent2,
                                                     DEMANDS, LoadCapacities, DISTANCES_FROM_DEPOTS,
                                                     DISTANCES_TO_DEPOTS, id_depot, crossoverRate)
                # if(not evaluate.haveEqualNodes(child1,child2,LOCATIONS)):
                #         print('note have Equal nodes, Crossover Bug!!!!!'+ str(gen))
                #         break
                #
                ## Mutation ##
                child1 = operation.mutate(child1, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                child2 = operation.mutate(child2, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                if (not evaluate.haveEqualNodes(child1, child2, NODES_OF_THIS_DEPOT)):
                    print('note have Equal nodes, Mutation Bug!!!!!' + str(gen))
                    break


                ## Evaluate Fitness of the best ones ###
                current_fitness = evaluate.chromosomeFitness(child1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                             DISTANCES_TO_DEPOTS, id_depot)
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(child2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))

                ## Add them back into populations ###
                populations.append(child1)
                populations.append(child2)
                populations.append(elite1)
                populations.append(elite2)

                ### Check break conditions ####
                if (current_fitness > best_fitness_so_far):
                    best_fitness_so_far = current_fitness
                    bestFitGen = gen
                    # print('#### New Best Fitness !! , Best so far is :' + str(10000.0 / best_fitness_so_far) + '#####')
                    # print('#### This Generation: ' + str(gen) + '#######')
                if (gen - bestFitGen >= 1000):
                    # print('#### Break Generation: ' + str(gen) + '#######')
                    break
                # print('############# Round-'+str(round+1) +',DEPOT-'+str(id_depot)+','+ 'Generation:' + str(gen + 1) + ' #########################')
                # print('This Gen Distance :' + str(10000.0 / current_fitness))

            ### Evaluate results of current DEPOT ####
            fitness_table = []
            for chromosome in populations:
                fitness_table.append(
                    evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot))
            populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
            # print("GA time --- %s seconds ---" % (time.time() - GA_time))
            best_chromosome = populations[0]
            computational_time = time.time() - calculation_time
            # print("Total Calculation time --- %s seconds ---" % (computational_time))
            distance = evaluate.chromosomeRoutesDistance(best_chromosome,DISTANCES,DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,id_depot)
            # print('Distances of the best chromosome: ' + str(distance))
            # print(best_chromosome)

            ## Memo the results ##
            results_all_depots.append(best_chromosome)
            distance_all_depots.append(distance)
            time_all_depots.append(computational_time)
        total_results.append((results_all_depots,distance_all_depots,time_all_depots))
        print(results_all_depots)
    return total_results

In [13]:
def mean_results(results):
    solutions = []
    cal_times = []
    distances = []
    for round, round_result in enumerate(results):
        sol = results[round][0]
        dist = results[round][1]
        ct = results[round][2]

        solutions.append(sol)
        distances.append(dist)
        cal_times.append(ct)
    distances = np.array(distances)
    distances = pd.DataFrame(distances)
    distances = np.mean(distances, axis=0)
    cal_times = np.array(cal_times)
    cal_times = pd.DataFrame(cal_times)
    cal_times = np.mean(cal_times, axis=0)
    return solutions,distances, cal_times


In [14]:
def create_worst_case(filepath, rounds = 1, population_size = 100, generations=3000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    # DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    DEPOT_NUMBERS = proc.worse2worst_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    
    res = GA.initialize_WorstCase_Chromosome(REQUESTS)
    
    
    dist_worst = 0
    for [idx,_,[p,d]] in res:
        depot = int(DEPOT_NUMBERS[idx]) 
        dist_worst += (DISTANCES_FROM_DEPOTS[depot][p]+DISTANCES[p][d]+DISTANCES_TO_DEPOTS[d][depot])

    return res,dist_worst

# Automated run

In [15]:
path = 'pdp_instances/GOO/'
FILENAMES = next(os.walk(path))[2]

all_results = []

for name in FILENAMES:
    filepath = path + name
    results = solve_and_print(filepath)
    solutions,distances,cal_times = mean_results(results)
    all_results.append((solutions,distances,cal_times))

----------------Instance name: pdp_instances/GOO/goo1.csv-----------------------------------
 processing time --- 0.5500640869140625 seconds ---
#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


...

[[[1, [13], [33, 30]], [0, [19, 18, 16, 15, 21, 14, 12], [45, 43, 42, 38, 37, 34, 39, 31, 36, 11, 67, 18, 14, 9]], [2, [22, 39, 28, 17, 20, 9], [44, 48, 32, 5, 61, 79, 40, 23, 7, 53, 74, 41]], [3, [29], [62, 80]]], [[0, [36, 25], [56, 72, 27, 28]]], [[3, [31, 7, 35, 2, 5, 0], [4, 70, 65, 64, 12, 6, 1, 21, 24, 20, 17, 2]], [2, [10, 3, 6, 1, 8, 32], [8, 25, 19, 13, 3, 22, 16, 66, 71, 49, 35, 46]], [1, [26, 38, 24], [57, 78, 55, 47, 26, 15]]], [], [[1, [34, 4, 11, 30, 27, 33], [29, 10, 63, 59, 54, 68, 69, 50, 60, 58, 73, 77]], [0, [23, 37], [76, 51, 52, 75]]]]
#################### Round :10 ############################


#################### Round :10 ############################


#################### Round :10 ############################


#################### Round :10 ############################


[[[3, [29], [62, 80]], [0, [21, 15, 12, 20, 19, 9, 13, 14, 16], [33, 44, 43, 45, 32, 30, 38, 37, 34, 31, 36, 39, 11, 23, 18, 14, 9, 41]], [1, [39, 22, 28, 17, 18], [48, 42, 5, 61, 79, 40, 7, 67, 53, 74]]], [[0, [36, 25], [56, 72, 27, 28]]], [[3, [10, 3, 6, 1, 8, 32], [8, 25, 19, 13, 3, 22, 16, 66, 71, 49, 35, 46]], [2, [38, 26, 24], [57, 78, 55, 47, 26, 15]], [1, [31, 35, 2, 7, 5, 0], [4, 70, 65, 64, 12, 6, 1, 21, 24, 20, 17, 2]]], [], [[1, [37, 23], [76, 51, 52, 75]], [0, [34, 30, 33, 11, 27, 4], [29, 10, 63, 59, 54, 68, 69, 50, 60, 58, 73, 77]]]]


# The GOO Instances

# Average list by columns

In [5]:
# filename = 'pdp_instances/GOO/goo1.csv'

In [6]:
# filepath = filename 
# filename = filename.rsplit('/', 1)[1]
# filename = filename.split('.')[0]
# print( 'Instance name : '+str(filename))


Instance name : goo1


In [7]:
# results = solve_and_print(filepath)

----------------Instance name: pdp_instances/GOO/goo1.csv-----------------------------------
 processing time --- 1.0620617866516113 seconds ---
#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


In [8]:
# results

[([[[2, [15], [35, 49]],
    [1, [19, 16], [41, 37, 34, 22]],
    [0, [13, 4, 6], [15, 31, 43, 18, 48, 42]]],
   [[1, [38], [79, 61]], [0, [39, 25, 24], [52, 51, 80, 21, 5, 10]]],
   [[6, [5, 3, 8], [9, 24, 16, 12, 11, 2]],
    [1, [21, 2, 12], [7, 45, 33, 30, 23, 25]],
    [5, [], []],
    [2, [32, 27, 11, 10, 33], [29, 63, 55, 77, 69, 28, 27, 64, 71, 65]],
    [4, [], []],
    [0, [31, 23, 7, 18], [62, 47, 40, 19, 20, 13, 50, 53]],
    [3, [9, 14, 0, 34], [32, 26, 3, 4, 67, 68, 66, 14]]],
   [[0, [30], [60, 58]]],
   [[3, [], []],
    [0, [17, 26, 22, 20], [54, 46, 44, 36, 39, 70, 78, 72]],
    [1, [29, 1, 28], [56, 57, 38, 1, 6, 59]],
    [2, [37, 35, 36], [74, 75, 73, 76, 17, 8]]]],
  [62342.0, 32954.0, 156140.0, 11241.0, 114938.0],
  [5.082298040390015,
   3.2432212829589844,
   23.598564386367798,
   1.1332643032073975,
   6.455474376678467])]

In [9]:
# res =results[0]
# SOL,DIST,CT = res[0],res[1],res[2]


In [6]:
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

In [47]:
worst_sol,distance_worst = create_worst_case('pdp_instances/GOO/goo10.csv')
distance_worst

727309.0

In [24]:
# SOL[2]
count_num_vehicle(SOL)

[3, 2, 5, 1, 3]

In [45]:
# DEPOT_NUMBERS

array([ 2.,  4.,  2.,  2.,  0.,  2.,  0.,  2.,  2.,  2.,  2.,  2.,  2.,
        0.,  2.,  0.,  0.,  4.,  2.,  0.,  4.,  2.,  4.,  2.,  1.,  1.,
        4.,  2.,  4.,  4.,  3.,  2.,  2.,  2.,  2.,  4.,  4.,  4.,  1.,  1.])

In [10]:
# results = [([[[2,[16,19],[41,37,34,22]],[1,[13,4,6],[15,31,43,18,48,42]],[0,[15],[35,49]]],[[1,[39,25,24],[52,51,80,21,5,10]],[0,[38],[79,61]]],[[2,[23,7,21],[19,47,45,50,53,25]],[5,[12,31,18],[62,40,30,23,20,13]],[3,[2,32,14,11,27,10,33],[32,7,4,29,33,63,55,77,69,28,27,64,71,65]],[1,[],[]],[0,[34,9,8,0],[3,26,24,68,67,66,2,14]],[4,[3,5],[9,16,12,11]]],[[0,[30],[60,58]]],[[3,[37,35,36],[74,75,73,76,17,8]],[2,[],[]],[1,[29,28,1],[56,57,38,1,6,59]],
                                                                                                                                                                                                                                                                                                                                                                                       [0,[26,22,20,17],[54,46,44,36,39,70,78,72]]]])]

In [16]:
# res = results[0]

In [18]:
# res[0]

[[2, [16, 19], [41, 37, 34, 22]],
 [1, [13, 4, 6], [15, 31, 43, 18, 48, 42]],
 [0, [15], [35, 49]]]

# Writing to CSV

In [16]:
solutions,distances,cal_times = mean_results(all_results)
dist = [dist for _,dist,_ in all_results]
dist = np.array(dist)
df = pd.DataFrame(dist)
df.to_csv('distances_goo.csv')
CT = [ct for _,_,ct in all_results]
CT = np.array(CT)
df_CT = pd.DataFrame(CT)
df_CT.to_csv('CT_goo.csv')

# Average list by columns

In [18]:
def avg_table(table):
    transposed = zip(*table)
    avg = lambda items: float(sum(items)) / len(items)
    averages = map(avg, transposed)
    averages = list(averages)
    return  averages

In [19]:
# for sol in solutions:
num_vehicles  = []
i = 0
for res in all_results:
    solutions = res[0]
    arr_temp = []
    for sol in solutions:
        num_temp = count_num_vehicle(sol)
        arr_temp.append(num_temp)
    avg_temp = avg_table(arr_temp)
    print(avg_temp)
    num_vehicles.append(avg_temp)

[3.0, 2.0, 4.4, 1.0, 3.0]
[3.8, 1.0, 3.7, 0.0, 2.7]
[3.0, 2.0, 3.0, 0.0, 3.0]
[2.8, 1.0, 3.7, 2.0, 2.0]
[2.0, 1.0, 3.0, 1.0, 2.0]
[3.1, 2.0, 3.0, 1.0, 3.0]
[2.1, 0.0, 4.0, 0.0, 3.8]
[2.0, 1.0, 2.0, 0.0, 3.0]
[2.0, 0.0, 3.2, 0.0, 2.0]
[3.1, 1.0, 3.0, 0.0, 2.0]


In [20]:

num_vehicles = np.array(num_vehicles)
df_num_vehicles = pd.DataFrame(num_vehicles)
df_num_vehicles.to_csv('num_veh_goo.csv')